<a href="https://colab.research.google.com/github/BhaskarFysik/AlgoShared/blob/main/bh_rsi_oscillator_and_smacross.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 5.1 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=85977562c3d30e29437502aa858d7a8d5800b619831f25f741fbfce50ca37ae4
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta


In [5]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [6]:
pip install backtesting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 5.0 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173832 sha256=2e63ece62d6b13cccd28ff8fd2945ab0eb8f0ef5fc9cc7fce91b9770dcbd5042
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting


In [7]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_ta as ta
#from backtesting.test import GOOG

symbol='SBIN'
df = yf.Ticker(f"{symbol}.NS").history(period="5y", interval="1d")
#df=pd.DataFrame(GOOG)
def RSI(values,n):
     return ta.rsi(close=df["Close"], length=n)
def SMA(values,n):
     return pd.Series(values).rolling(n).mean()



class RsiOscillator(Strategy):
    upper_bound=70
    lower_bound=30
    n1=20
    n2=50
    def init(self):
        self.rsi=self.I(RSI,self.data.Close,14)
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    def next(self):

        if crossover(self.rsi,self.upper_bound) and self.sma2>self.sma1:
            self.position.close()
        elif crossover(self.lower_bound,self.rsi) and self.sma2<self.sma1:
            self.buy()

bt=Backtest(df,RsiOscillator, cash=10000)
stats=bt.optimize(
    upper_bound=range(10,85,5),
    lower_bound=range(10,85,5),
    n1=range(5, 30, 5),
    n2=range(10, 70, 5),
    maximize='Equity Final [$]',
    constraint=lambda param: param.upper_bound>param.lower_bound and param.n1 < param.n2)
#stats=bt.run()
print(stats)
bt.plot()

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.7/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 5250 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/18 [00:00<?, ?it/s]

Start                     2017-10-16 00:00...
End                       2022-10-14 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                   84.061489
Equity Final [$]                 20824.990334
Equity Peak [$]                  22561.539895
Return [%]                         108.249903
Buy & Hold Return [%]               42.715012
Return (Ann.) [%]                   16.132609
Volatility (Ann.) [%]               34.915065
Sharpe Ratio                         0.462053
Sortino Ratio                        0.799085
Calmar Ratio                         0.405748
Max. Drawdown [%]                  -39.760179
Avg. Drawdown [%]                   -5.734476
Max. Drawdown Duration      337 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                   11
Win Rate [%]                        81.818182
Best Trade [%]                      50.569384
Worst Trade [%]                    -14.917027
Avg. Trade [%]                    

Row(id='1500', ...)